In [60]:
import gensim
import numpy
import logging
import keras
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Dense
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
#from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
#from sklearn.model_selection import cross_val_score
#from sklearn.model_selection import KFold
#from sklearn.preprocessing import LabelEncoder
#from sklearn.pipeline import Pipeline
from itertools import zip_longest, count
import json 
import nltk
from keras.models import Sequential
from keras.layers.core import Dense, Activation, Merge, Dropout, Reshape, RepeatVector, Masking
from keras.layers.recurrent import LSTM,GRU
from collections import Counter
from keras.layers.wrappers import TimeDistributed

In [61]:
# Load Files into Memory.

image_features = numpy.load('smaller_merged_train.npy')
with open ('smaller_merged_train.json') as f:
    image_captions = json.load(f)

# Remove this to end fixed testing with 5 data features
image_features = image_features[:5]

In [62]:
# Generates a list containing all sentences appended with start and termination symbols.
# select_list contains the selector indexed sentence.

input_list = []
select_list = []
selector = 0
for i in range(len(image_captions)):
    sent_temp = []
    for j in range(len(image_captions[i][1])):
        temp = ['<s>']
        temp.extend(image_captions[i][1][j])
        temp.append("</s>")        
        input_list.append(temp)
        if (j==selector):
            select_list.append(temp)

# Remove this to end fixed testing with 5 sentences
select_list = select_list[:5]

In [63]:
# Generation of nb_samples parameter 

nb_samples = len(select_list)

# Generation of Dictionary and vocab_size parameter

word_dict = []

for i in range(len(select_list)):
    for j in range(len(select_list[i])):
        if (select_list[i][j] not in word_dict):
            word_dict.append(select_list[i][j])

# Vocab size increased by 1 to avoid 0 as an integer label.

vocab_size = len(word_dict) + 1

print("Size of this Vocabulary is %r" %vocab_size) 

#Generation of MAXLEN Parameter            

MAXLEN = 0

for i in range(len(select_list)):
    if (len(select_list[i])>MAXLEN):
        MAXLEN = len(select_list[i])

print("Maxlen for this Data is %r "%MAXLEN)            


Size of this Vocabulary is 46
Maxlen for this Data is 18 


In [76]:
char_labels = {ch:i for i, ch in enumerate(word_dict)}

labels_char = {i:ch for i, ch in enumerate(word_dict)}

In [65]:
# Generate Input and Output Streams

x_train = []
y_train = []

for i in range(len(select_list)):
    x_train.append(select_list[i][:-1])
    y_train.append(select_list[i][1:])

x_train = numpy.asarray(x_train)


#Convert Streams to their Integer valued counterparts

for i in range(len(x_train)):
    for j in range(len(x_train[i])):
        x_train[i][j] = char_labels[x_train[i][j]] + 1
        
x_train = numpy.asarray(x_train)


x_train = sequence.pad_sequences(x_train,maxlen = MAXLEN, padding = 'post')

        
for i in range(len(y_train)):
    for j in range(len(y_train[i])):
        y_train[i][j] = char_labels[y_train[i][j]]
        
y_train = sequence.pad_sequences(y_train,maxlen = MAXLEN, padding = 'post')

# Convert Outputs to One-Hot form

y_out = numpy.zeros((nb_samples,MAXLEN,vocab_size))

for i in range(len(y_train)):
    for j in range(len(y_train[i])):
        y_out[i][j][y_train[i][j]] = 1

y_train = y_out



In [52]:
num_hidden_units_mlp = 1024
image_model = Sequential()
image_model.add(Dense(128,input_dim = 4096))

language_model = Sequential()
language_model.add(Embedding(vocab_size, 256, input_length=MAXLEN))
language_model.add(GRU(output_dim=128, return_sequences=True))
language_model.add(TimeDistributed(Dense(128)))

image_model.add(RepeatVector(MAXLEN))

model = Sequential()
model.add(Merge([image_model, language_model], mode='concat', concat_axis=-1))

model.add(GRU(256, return_sequences=True))

model.add(TimeDistributed(Dense(vocab_size)))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy', optimizer='rmsprop')


Epoch 1/40
5/5 [==============================] - 0s - loss: 4.1915     
Epoch 2/40
5/5 [==============================] - 0s - loss: 3.4773     
Epoch 3/40
5/5 [==============================] - 0s - loss: 2.9522     
Epoch 4/40
5/5 [==============================] - 0s - loss: 2.6705     
Epoch 5/40
5/5 [==============================] - 0s - loss: 2.4877     
Epoch 6/40
5/5 [==============================] - 0s - loss: 2.3470     
Epoch 7/40
5/5 [==============================] - 0s - loss: 2.1759     
Epoch 8/40
5/5 [==============================] - 0s - loss: 2.0980     
Epoch 9/40
5/5 [==============================] - 0s - loss: 2.1421     
Epoch 10/40
5/5 [==============================] - 0s - loss: 2.0353     
Epoch 11/40
5/5 [==============================] - 0s - loss: 1.8956     
Epoch 12/40
5/5 [==============================] - 0s - loss: 1.8774     
Epoch 13/40
5/5 [==============================] - 0s - loss: 1.8469     
Epoch 14/40
5/5 [==============================

In [ ]:
# Keep best weights and save callback history

# define the checkpoint

filepath="weights-improvement-{epoch:02d}-{loss:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

In [54]:
model.fit([image_features, x_train], y_train, batch_size=1, nb_epoch=40)

Epoch 1/40
5/5 [==============================] - 0s - loss: 0.9420     
Epoch 2/40
5/5 [==============================] - 0s - loss: 0.9325     
Epoch 3/40
5/5 [==============================] - 0s - loss: 0.9050     
Epoch 4/40
5/5 [==============================] - 0s - loss: 0.9448     
Epoch 5/40
5/5 [==============================] - 0s - loss: 0.9213     
Epoch 6/40
5/5 [==============================] - 0s - loss: 0.8966     
Epoch 7/40
5/5 [==============================] - 0s - loss: 0.8522     
Epoch 8/40
5/5 [==============================] - 0s - loss: 0.8396     
Epoch 9/40
5/5 [==============================] - 0s - loss: 0.8465     
Epoch 10/40
5/5 [==============================] - 0s - loss: 0.8596     
Epoch 11/40
5/5 [==============================] - 0s - loss: 0.8695     
Epoch 12/40
5/5 [==============================] - 0s - loss: 0.9199     
Epoch 13/40
5/5 [==============================] - 0s - loss: 0.8427     
Epoch 14/40
5/5 [==============================

In [86]:
print (image_features[1],x_train[1])
prediction = model.predict([image_features, x_train], verbose=0)
for i in range(len(prediction)):
    pred_sentence = []
    for j in range(len(prediction[i])):
        index = numpy.argmax(prediction[i][j])
        if index > 0:
            index -= 1
        word = labels_char[index]
        pred_sentence.append(word)
    print (pred_sentence)

[ 7.04522991  0.99864912  0.         ...,  0.          0.          4.49929237] [ 1 11 12 13 14 15 16 17 18  4 17 19 20  0  0  0  0  0]
['<s>', 'Two', 'people', 'in', 'dim', 'lit', 'room', 'playing', 'Nintendo', '<s>', '<s>', '<s>', '<s>', '<s>', '<s>', '<s>', '<s>', '<s>']
['</s>', '</s>', 'white', 'sink', 'next', 'next', 'a', 'to', 'to', 'to', 'to', 'toilet', 'Nintendo', '<s>', '<s>', '<s>', '<s>', '<s>']
['man', 'man', 'man', 'man', 'paddle', 'surfing', 'to', 'to', 'to', 'to', 'blue', 'blue', 'blue', 'bathroom', 'bathroom', 'Nintendo', 'Nintendo', 'Nintendo']
['</s>', '</s>', 'bunk', 'bed', 'that', 'has', 'three', 'bunks', 'bunks', 'Nintendo', '<s>', '<s>', '<s>', '<s>', '<s>', '<s>', '<s>', '<s>']
['<s>', 'it', 'young', 'girls', 'girls', 'to', 'flying', 'flying', 'kite', 'bathroom', 'bathroom', 'Nintendo', '<s>', '<s>', '<s>', '<s>', '<s>', '<s>']
